In [1]:
import numpy as np
import pandas as pd

In [122]:
class GameTree_Kuhn2P:
    def __init__(self):
        self.gameTree = {0: {"name": "START", 
                             "thisPlayer": None, "nextPlayer": "p0",
                             "thisRound": 0, "nextRound": 1,
                             "conn_up": None, "conn_down": []}}
        self.playerActions = {"p0": [],
                              "p1": []}
        
        self.addRound_1()
        self.addRound_2()
        self.addRound_3()
       
    # ----------------------------------------------------------------------------------------------------
    #          first round
    # ----------------------------------------------------------------------------------------------------
    def addRound_1(self):
        thisRound, nextRound = 1, 2
        nodeUp  = 0
        
        nodeNew = 1
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "check", 
                                  "thisPlayer": "p0", "nextPlayer": "p1", 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": []}
        self.playerActions["p0"].append(nodeNew)
        
        nodeNew = 2
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "bet", 
                                  "thisPlayer": "p0", "nextPlayer": "p1", 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": []}
        self.playerActions["p0"].append(nodeNew)
        
    # ----------------------------------------------------------------------------------------------------
    #          second round
    # ----------------------------------------------------------------------------------------------------  
    def addRound_2(self):
        thisRound, nextRound = 2, 3
        
        # responses to P1:check
        nodeUp  = 1
        nodeNew = 3
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "check", 
                                  "thisPlayer": "p1", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": None,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "SHOWDOWN",
                                  "winAmount": 1}
        self.playerActions["p1"].append(nodeNew)
        
        nodeNew = 4
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "bet", 
                                  "thisPlayer": "p1", "nextPlayer": "p0", 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": []}
        self.playerActions["p1"].append(nodeNew)
        
        # responses to P1:bet
        nodeUp  = 2
        nodeNew = 5
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "fold", 
                                  "thisPlayer": "p1", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": None,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "p0",
                                  "winAmount": 1}
        self.playerActions["p1"].append(nodeNew)
        
        nodeNew = 6
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "call", 
                                  "thisPlayer": "p1", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": None,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "SHOWDOWN",
                                  "winAmount": 2}
        self.playerActions["p1"].append(nodeNew)
     
    # ----------------------------------------------------------------------------------------------------
    #          third round
    # ----------------------------------------------------------------------------------------------------
    def addRound_3(self):
        thisRound, nextRound = 3, None
        
        # responses to P2:bet
        nodeUp  = 4
        nodeNew = 7
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "fold", 
                                  "thisPlayer": "p0", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "p1",
                                  "winAmount": 1}
        self.playerActions["p0"].append(nodeNew)
        
        nodeNew = 8
        self.gameTree[nodeUp]["conn_down"].append(nodeNew)
        self.gameTree[nodeNew] = {"name": "call", 
                                  "thisPlayer": "p0", "nextPlayer": None, 
                                  "thisRound": thisRound, "nextRound": nextRound,
                                  "conn_up": nodeUp, "conn_down": [],
                                  "winningPlayer": "SHOWDOWN",
                                  "winAmount": 2}
        self.playerActions["p0"].append(nodeNew)

In [123]:
gt = GameTree_Kuhn2P()

In [124]:
gt.playerActions

{'p0': [1, 2, 7, 8], 'p1': [3, 4, 5, 6]}

In [134]:
# strategy description
# strategy = {
#   "hand": $currentNode$: {action1: prob(action1), action2: prob(action2)}}

# 3 baseLine strategies for player-1 that can be applied to each hand
#  two actions are possible, depending on what p1 chose if p0 checks
#   --> strat_p0_CheckFold, strat_p0_CheckCall, strat_p0_Bet
strat_p0_CheckFold = {0: {1: 1.0, 2: 0.0} , 4: {7: 1.0, 8: 0.0}}
strat_p0_CheckCall = {0: {1: 1.0, 2: 0.0} , 4: {7: 0.0, 8: 1.0}}
strat_p0_Bet       = {0: {1: 0.0, 2: 1.0} , 4: None}

strategy_p0 = {
    "J": strat_p0_CheckFold,
    "Q": strat_p0_CheckCall,
    "K": strat_p0_Bet}

# 4 baseLine strategies for player-2 that can be applied to each hand
#  two actions are possible, depending on what p0 chose as first action (check or bet)
#  --> strat_p1_CheckFold, strat_p1_CheckCall, strat_p1_BetFold, strat_p1_BetCall
strat_p1_CheckFold = {1: {3: 1.0, 4: 0.0} , 2: {5: 1.0, 6: 0.0}}
strat_p1_CheckCall = {1: {3: 1.0, 4: 0.0} , 2: {5: 0.0, 6: 1.0}}
strat_p1_BetFold   = {1: {3: 0.0, 4: 1.0} , 2: {5: 1.0, 6: 0.0}}
strat_p1_BetCall   = {1: {3: 0.0, 4: 1.0} , 2: {5: 0.0, 6: 1.0}}

strategy_p1 = {
    "J": strat_p1_CheckFold,
    "Q": strat_p1_CheckCall,
    "K": strat_p1_BetCall}

strategy = {
    "p0": strategy_p0,
    "p1": strategy_p1}

In [137]:
# hand values:
hand_values = {"J": 1, "Q": 2, "K": 3}

# chose hands for players
hand = {
    "p0": "K",
    "p1": "Q"}

currentNode = 0
gameOver = False

dict_gamePlayed = {}

while not gameOver:
    list_nextNodes = gt.gameTree[currentNode]["conn_down"]
    nextPlayer = gt.gameTree[currentNode]["nextPlayer"]
    thisRound = gt.gameTree[currentNode]["thisRound"]
    nextRound = gt.gameTree[currentNode]["nextRound"]
    
    # identify relevant strategy for player
    playerStrategy = strategy[nextPlayer]
    playerHand = hand[nextPlayer]
    
    # decide actions to take
    possible_actions = playerStrategy[playerHand][currentNode]

    arr_actions, arr_prob = np.array([]), np.array([]) 
    for action, prob in possible_actions.items():
        if action not in list_nextNodes:
            raise ValueError("One of the actions ({}) was not found in game-tree".format(action))

        arr_actions = np.append(arr_actions, action)
        arr_prob = np.append(arr_prob, prob)

    # pick action based on probability distribution
    chosen_action = int(np.random.choice(arr_actions, size=1, p=arr_prob))
    dict_gamePlayed[nextRound] = {"player": nextPlayer, "action": chosen_action}

    # move to next round
    currentNode = chosen_action
    
    # finish game if over
    if gt.gameTree[currentNode]["nextRound"] is None:
        gameOver = True
        
        # evaluate payoff
        winningPlayer = gt.gameTree[currentNode]["winningPlayer"]
        if winningPlayer == "SHOWDOWN":
            if hand_values[hand["p0"]] > hand_values[hand["p1"]]:
                winningPlayer = "p0"
            elif hand_values[hand["p0"]] < hand_values[hand["p1"]]:
                winningPlayer = "p1"
        winAmount = gt.gameTree[currentNode]["winAmount"]
            
print(f"{winningPlayer} won. winAmount={winAmount}")

p0 won. winAmount=2


In [111]:
dict_gamePlayed

{1: {'player': 'p0', 'action': 2}, 2: {'player': 'p1', 'action': 6}}

In [70]:
np.random.choice([1,2,3], size=1, p=[0.1, 0.9, 0.0])

array([2])